# Neural Collaborative Filtering

In [3]:
import torch
from torch import nn, optim
import numpy as np

# ユーザー数、アイテム数、評価値の範囲
num_users = 50
num_items = 50
max_rating = 5
min_rating = 1

# データセットの生成
ratings = np.random.randint(min_rating, max_rating+1, size=(num_users, num_items))
ratings = (ratings-1)/4

# ネットワークの定義
class NCF(nn.Module):
    def __init__(self, num_users, num_items, factor_num=5, layers=[10,5]):
        super(NCF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = factor_num
        self.layers = layers

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=5)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=5)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating


In [4]:
model = NCF(num_users, num_items)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [5]:
losses = []
for epoch in range(10):
    model.train() # Enable dropout (if have).
    
    epoch_loss = 0.0
    for user_index in range(num_users):
        for item_index in range(num_items):
            rating = float(ratings[user_index, item_index])
            label = torch.FloatTensor([[rating]])
        
            optimizer.zero_grad()
            user = torch.LongTensor([user_index])
            item = torch.LongTensor([item_index])
            prediction = model(user, item)
            
            loss = loss_function(prediction, label)
            loss.backward()
            optimizer.step()
            epoch_loss+=loss.item()
    
    losses.append(epoch_loss/(num_users*num_items))
    model.eval()

In [6]:
losses

[0.6988408488035202,
 0.6969971101880074,
 0.6958758408546448,
 0.6950538464546203,
 0.6943823150038719,
 0.6938250337600708,
 0.693351780462265,
 0.6929443723917007,
 0.6925905404210091,
 0.6922729956626892]